In [72]:
import pandas as pd
import numpy as np

# Earth Engine Python client library
import ee
import geemap.core as geemap

# Resterio to load files
import rasterio
from rasterio.enums import Resampling

import tensorflow as tf
from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split

import csv

In [73]:
# Earth Engine Python client library config
# Authenticate (redirect to auth page if not)
ee.Authenticate()

True

In [26]:
# Get Google project
ee.Initialize(project = 'ee-nikolaydragomirovzhechev')

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterDate('2022-01-01', '2022-01-31')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

rgb_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(-9.1695, 38.6917, 12)
m.add_layer(dataset.median(), rgb_vis, 'RGB')
m

## Landsat surface dataset

ref: https://www.usgs.gov/landsat-missions

#### Filter, process, and prepare the dataset in Google Earth Engine

In [41]:
# Load the Landsat surface reflectance data
# Continuous Change Detection and Classification (CCDC) algorithm
dataset_Landsat = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")

longitude_min: The minimum longitude (western boundary) of your AOI.  
latitude_min: The minimum latitude (southern boundary) of your AOI.  
longitude_max: The maximum longitude (eastern boundary) of your AOI.  
latitude_max: The maximum latitude (northern boundary) of your AOI.

In [43]:
# Coordinates for California
longitude_min = -114.2579
latitude_min = 38.9275
longitude_max = -115
latitude_max = 42

# Define the area of interest (AOI) and time range
# aoi = ee.Geometry.Rectangle([longitude_min, latitude_min, longitude_max, latitude_max])

aoi_urban = ee.Geometry.Rectangle([-119.7, 34.5, -119.0, 35.0])  # Example: urban area
aoi_forest = ee.Geometry.Rectangle([-120.7, 35.5, -120.0, 36.0])  # Example: forest area

start_date = '2022-01-01'
end_date = '2023-01-01'

# Create a dictionary to map regions to labels
regions_labels = [
    {"region": aoi_urban, "label": "urban"},
    {"region": aoi_forest, "label": "forest"},
    # Add more regions and labels as needed
]

In [46]:
dataset_filtered = dataset_Landsat.filterDate(start_date, end_date).filterBounds(aoi_urban)
print(dataset_filtered.size().getInfo())

45


NDVI info:  
https://gisgeography.com/ndvi-normalized-difference-vegetation-index/

$ NDVI = \frac{(NIR−RED)}{(NIR+RED)} $

In [54]:
def compute_ndvi(image):
    # ndvi = image.normalizedDifference(['NIR', 'RED']).rename('NDVI')
    ndvi = image.normalizedDifference(['SR_B7', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

Bands provide information about different spectral characteristics derived from Landsat satellite imagery.  
Key Bands in the Dataset:
Blue (B1):

Wavelength: ~450-520 nm
Useful for: Coastal and aerosol studies, distinguishing soil from vegetation, and monitoring water bodies.
Green (B2):

Wavelength: ~520-600 nm
Useful for: Assessing vegetation vigor, distinguishing between different types of vegetation, and monitoring aquatic systems.
Red (B3):

Wavelength: ~630-690 nm
Useful for: Discriminating between vegetation types, monitoring crop health, and assessing urban vs. rural areas.
Near-Infrared (NIR, B4):

Wavelength: ~770-900 nm
Useful for: Vegetation health, crop monitoring, and distinguishing between water bodies and land.
Shortwave Infrared 1 (SWIR1, B5):

Wavelength: ~1550-1750 nm
Useful for: Assessing vegetation moisture, soil moisture, and differentiating between snow, clouds, and other features.
Shortwave Infrared 2 (SWIR2, B6):

Wavelength: ~2080-2350 nm
Useful for: Monitoring droughts, soil moisture, and differentiating between snow/ice and clouds.
Thermal Infrared (TIR, B10):

Wavelength: ~10.4-12.5 µm
Useful for: Monitoring temperature, surface heat emissions, and identifying hot spots (e.g., wildfires).
Special Bands from the CCDC Algorithm:
Trend Coefficients (Slope and Intercept):

These bands represent the trend of surface reflectance values over time, generated by the CCDC algorithm. The slope indicates whether the reflectance has increased or decreased, while the intercept gives the baseline value at the beginning of the time series.
Seasonality Components:

CCDC decomposes time-series data into seasonal components, capturing cyclic patterns such as seasonal vegetation growth.
Breakpoint Count:

This band records the number of "breakpoints" where land cover change was detected by the CCDC algorithm over the 1999-2019 period.
Segment Number:

Each pixel in the CCDC dataset is divided into segments based on change over time. The segment number indicates which part of the time series the pixel belongs to (e.g., a stable segment or a transition due to land cover change).

In [53]:
dataset_with_ndvi = dataset_filtered.map(compute_ndvi)

# Get a median composite of the NDVI band
ndvi_composite = dataset_with_ndvi.select('NDVI').median()

# Visualize the result
m = geemap.Map()
m.centerObject(aoi, 10)
m.addLayer(ndvi_composite, {'min': 0, 'max': 1, 'palette': ['white', 'green', 'brown']}, 'NDVI')
m

Map(center=[40.452629539490076, -114.62895000000017], controls=(ZoomControl(options=['position', 'zoom_in_text…

#### Export Data to a Machine Learning-Compatible Format

In [61]:
dataset_with_ndvi

In [62]:
def export ():
    # export task
    task = ee.batch.Export.image.toDrive(
        image = ndvi_composite,
        description = 'Landsat_NDVI_Export_01',
        scale = 30, # The resolution of the image in meters
        region = aoi,
        fileFormat = 'GeoTIFF'
    )
    
    task.start()

Load exported results.

In [55]:
 # Open the exported GeoTIFF
with rasterio.open('Datasets/Landsat/Landsat_NDVI_Export_01.tif') as src:
    # Read all bands into a NumPy array
    image_data = src.read()  # Shape: (bands, height, width)

    # Select the NDVI band (assuming it's stored in one of the bands)
    ndvi_band = src.read(1)  # Replace '1' with the correct band number if necessary

    # Normalize NDVI band between 0 and 1
    ndvi_normalized = (ndvi_band - ndvi_band.min()) / (ndvi_band.max() - ndvi_band.min())

    # If needed, convert the image data from a 3D to 4D format (e.g., for CNN input)
    image_data = np.transpose(image_data, (1, 2, 0))  # Shape: (height, width, bands)

    # If using CNNs, we need to reshape or expand dimensions (from 2D to 3D)
    X = np.expand_dims(ndvi_normalized, axis=-1)  # Add channel dimension -> shape (height, width, 1)

    # Resize the image to 256x256 (if required for your model)
    X_resized = np.resize(X, (256, 256, 1))  # Shape: (256, 256, 1)

    # Get the image size (width, height)
    width = src.width
    height = src.height
    num_channels = src.count  # Number of bands

    # Define the new dimensions (e.g., 256x256 pixels)
    new_width, new_height = 256, 256
    scale_factor_x = new_width / width
    scale_factor_y = new_height / height

    # Resize the image using the 'average' resampling method
    resized_img = src.read(
        out_shape=(src.count, new_height, new_width),  # (num_bands, height, width)
        resampling=Resampling.average
    )

    # Save the resized image
    with rasterio.open('resized_image.tif', 'w', 
                       driver=src.driver, 
                       width=new_width, 
                       height=new_height, 
                       count=src.count, 
                       dtype=src.dtypes[0], 
                       crs=src.crs, 
                       transform=src.transform) as dst:
        dst.write(resized_img)

In [58]:
ndvi_band

array([[-0.08863541, -0.08684293, -0.08619867, ..., -0.11585655,
        -0.11169079, -0.08576661],
       [-0.08606752, -0.08865455, -0.08852252, ..., -0.11155718,
        -0.10787684, -0.07197849],
       [-0.08496384, -0.09270717, -0.08482006, ..., -0.10102294,
        -0.09201819, -0.0609365 ],
       ...,
       [-0.05845913, -0.05852353, -0.05852353, ..., -0.11820606,
        -0.13110834, -0.12617202],
       [-0.03963536, -0.05901128, -0.05901128, ..., -0.11439221,
        -0.11688019, -0.12049704],
       [-0.04585337, -0.03990377, -0.03990377, ..., -0.11710235,
        -0.11451808, -0.11666176]], dtype=float32)

#### Export images with labels

In [59]:
# Create a CSV file for labels
with open('image_labels.csv', 'w', newline = '') as csvfile:
    fieldnames = ['region_id', 'label', 'min_lon', 'min_lat', 'max_lon', 'max_lat']
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
    
    writer.writeheader()
    
    # Loop through each region and export images along with labels
    for idx, item in enumerate(regions_labels):
        region = item["region"]
        label = item["label"]
        
        # Export task for imagery
        export()

        # Extract region coordinates (bounds) and save to CSV
        coords = region.bounds().coordinates().get(0).getInfo()  # Get the bounding box
        min_lon, min_lat = coords[0]
        max_lon, max_lat = coords[2]
        
        # Write label and coordinates to CSV
        writer.writerow({
            'region_id': idx,
            'label': label,
            'min_lon': min_lon,
            'min_lat': min_lat,
            'max_lon': max_lon,
            'max_lat': max_lat
        })

print("CSV file for labels has been created.")

CSV file for labels has been created.


#### Convolutional Neural Network (CNN) model for classification  
https://www.tensorflow.org/tutorials/images/cnn

In [63]:
image_data_csv = pd.read_csv('image_labels.csv')

In [68]:
# Define model
height = 256
width = 256
num_channels_input = num_channels # depends on the image bands
num_classes = 2 # number of distinct categories or classes (e.g., distinguishing between two classes, like "urban" vs. "rural")

model = models.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (height, width, num_channels_input)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation ='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation ='relu'),
        layers.Flatten(),
        layers.Dense(64, activation = 'relu'),
        layers.Dense( num_classes, activation = 'softmax')  # For classification
    ]
)

# Compile [configure] the model
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

X = np.expand_dims(ndvi_normalized, axis=-1)
y = image_data_csv['label'].values
y_expanded = np.concatenate([np.repeat(0, X.shape[0] // 2), np.repeat(1, X.shape[0] // 2)])

# Split data into training and validation sets (e.g., 80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y_expanded, test_size=0.2, random_state=42)

# Train the model (assuming X_train is the image data and y_train are labels)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

C:\Users\zhech\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Found input variables with inconsistent numbers of samples: [11437, 11436]

In [71]:
print(f"Number of samples in X: {X.shape[0]}")
print(f"Number of samples in y: {y_expanded.shape[0]}")

Number of samples in X: 11437
Number of samples in y: 11436
